In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:\machihe learning\Projects\Recomendation system\src\products.csv')

In [3]:
df.head(3)

,product_id,product_name,product_category,product_description,price
0,1,Follow Power,Toys & Games,Upon improve must put. Billion network left ac...,350.24
1,2,International Mr,Toys & Games,War three toward charge. Society event into me...,81.60
2,3,However Fish,Books,Arrive investment stage firm toward stay. Prod...,495.15


In [4]:
# Load the datasets
try:
    interactions_df = pd.read_csv('C:\\machihe learning\\Projects\\Recomendation system\\src\\interactions.csv')
    products_df = pd.read_csv('C:\\machihe learning\\Projects\\Recomendation system\\src\\products.csv')
    users_df = pd.read_csv('C:\\machihe learning\\Projects\\Recomendation system\\src\\users.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: One or more CSV files not found. Make sure they are in the correct directory.")

Data loaded successfully.


In [5]:
# --- Perform initial inspection ---
print("\nInteractions DataFrame Info:")
interactions_df.info()


Interactions DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     100000 non-null  int64 
 1   product_id  100000 non-null  int64 
 2   rating      100000 non-null  int64 
 3   timestamp   100000 non-null  object
dtypes: int64(3), object(1)
memory usage: 3.1+ MB


In [6]:
# --- Merge the DataFrames ---
# We'll merge interactions with products to get product details
merged_df = pd.merge(interactions_df, products_df, on='product_id')

In [7]:
# Now, we'll merge with users to get user details
final_df = pd.merge(merged_df, users_df, on='user_id')

In [8]:
interactions_with_ratings_df = final_df.copy()

In [9]:
import os
os.makedirs('data/processed', exist_ok=True)
final_df.to_csv('data/processed/final_df.csv', index=False)
print('final_df saved to data/processed/final_df.csv')

final_df saved to data/processed/final_df.csv


In [10]:
print("\nFinal Merged DataFrame Info:")
final_df.info()


Final Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              100000 non-null  int64  
 1   product_id           100000 non-null  int64  
 2   rating               100000 non-null  int64  
 3   timestamp            100000 non-null  object 
 4   product_name         100000 non-null  object 
 5   product_category     100000 non-null  object 
 6   product_description  100000 non-null  object 
 7   price                100000 non-null  float64
 8   signup_date          100000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 6.9+ MB


In [11]:
import os

os.makedirs('data/processed', exist_ok=True)
final_df.to_csv('data/processed/final_df.csv', index=False)

In [12]:
final_df.head(3)

,user_id,product_id,rating,timestamp,product_name,product_category,product_description,price,signup_date
0,279,1448,1,2024-01-01 07:47:50,Full Act,Apparel,Including experience just side. Meet factor in...,132.54,2023-12-08
1,385,1906,1,2024-01-12 19:14:44,Medical Catch,Sports,Test former major all. Once support political....,361.64,2024-05-23
2,740,2239,1,2024-05-11 08:04:50,Machine Exactly,Sports,Himself citizen mother. Age born south already...,340.67,2021-08-29


# Build the model

In [13]:
pip install scikit-surprise

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\machihe learning\Projects\Recomendation system\recom39\Scripts\python.exe -m pip install --upgrade pip' command.


In [14]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [15]:
!cd

c:\machihe learning\Projects\Recomendation system\notebooks


In [16]:
# Load the processed data
purchases_df = pd.read_csv('data/processed/final_df.csv')

In [17]:
reader = Reader(rating_scale=(1, 5)) 

In [18]:
# Load the DataFrame into the Surprise Dataset format
data = Dataset.load_from_df(purchases_df[['user_id', 'product_id', 'rating']], reader)

# Train the model

In [19]:
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

print("Model training complete.")

Model training complete.


In [20]:
# --- 3. Make a prediction for a specific user ---

In [21]:
# Let's pick a random user to make a prediction for.
user_id_to_predict = purchases_df['user_id'].iloc[0] # Get the first user from the DataFrame
product_id_to_predict = products_df[~products_df['product_id'].isin(
    purchases_df[purchases_df['user_id'] == user_id_to_predict]['product_id']
)]['product_id'].iloc[0]

# Predict the rating the user would give to the product
prediction = model.predict(user_id_to_predict, product_id_to_predict)

print(f"\nPredicted rating for user {user_id_to_predict} on product {product_id_to_predict}: {prediction.est}")


Predicted rating for user 279 on product 1: 1.8301754026202681


In [22]:
# --- 4. Get Top N recommendations for a user ---
# This is a more practical application.
def get_top_n_recommendations(model, user_id, all_products, k=10):
    # Get products the user has NOT purchased
    user_purchased_items = purchases_df[purchases_df['user_id'] == user_id]['product_id']
    unseen_products = [product for product in all_products if product not in user_purchased_items.values]

    # Predict ratings for all unseen products
    predictions = [model.predict(user_id, product_id) for product_id in unseen_products]

    # Sort the predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top k products
    top_k_products = predictions[:k]
    
    return [pred.iid for pred in top_k_products]

# Get all unique product IDs
all_products = products_df['product_id'].unique()

top_recommendations = get_top_n_recommendations(model, user_id_to_predict, all_products)
print(f"\nTop 10 recommended product IDs for user {user_id_to_predict}:")
print(top_recommendations)


Top 10 recommended product IDs for user 279:
[2776, 2747, 4097, 3751, 597, 3252, 824, 3680, 4604, 1800]


# Evaluating

In [23]:
# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [24]:
# --- Train the model on the training set ---
model = SVD()
model.fit(trainset)

In [25]:
# --- Make predictions on the testing set ---
predictions = model.test(testset)

In [26]:
rmse = accuracy.rmse(predictions)
print(f"The model's RMSE on the test set is: {rmse}")

RMSE: 1.3907
The model's RMSE on the test set is: 1.3906842219969462


In [27]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions."""
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=10)

In [28]:
# Get all unique product IDs for recommendations
all_products = products_df['product_id'].unique()

In [29]:
# Choose a user to make recommendations for (e.g., the first user in the DataFrame)
user_id_to_predict = interactions_with_ratings_df['user_id'].iloc[0]

In [30]:
# Get the top 10 recommended product IDs
top_recommendations = get_top_n(model.test(trainset.build_anti_testset()), n=10)[user_id_to_predict]

In [31]:
print(f"\nTop 10 recommended product IDs and estimated ratings for user {user_id_to_predict}:")
for product_id, est_rating in top_recommendations:
    print(f"Product ID: {product_id}, Estimated Rating: {est_rating:.4f}")


Top 10 recommended product IDs and estimated ratings for user 279:
Product ID: 4612, Estimated Rating: 3.2568
Product ID: 134, Estimated Rating: 3.1847
Product ID: 581, Estimated Rating: 3.1395
Product ID: 4614, Estimated Rating: 3.1010
Product ID: 1894, Estimated Rating: 3.0749
Product ID: 4101, Estimated Rating: 3.0748
Product ID: 4711, Estimated Rating: 3.0653
Product ID: 2488, Estimated Rating: 3.0471
Product ID: 1093, Estimated Rating: 2.9975
Product ID: 4306, Estimated Rating: 2.9887


In [32]:
from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=4):
    """
    Return precision and recall at k metrics for each user.
    'threshold' is the rating above which an item is considered 'relevant'
    for the user. We'll set it to 4 to consider only 'purchase' events as relevant.
    """
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = defaultdict(float)
    recalls = defaultdict(float)
    
    for uid, user_ratings in user_est_true.items():
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_relevant = sum(1 for _, true_r in user_ratings if true_r >= threshold)

        # Number of recommended items in top k
        n_rec_k = sum(1 for est, _ in user_ratings[:k] if est >= threshold)

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(1 for est, true_r in user_ratings[:k] if true_r >= threshold and est >= threshold)

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_relevant if n_relevant != 0 else 0

    return precisions, recalls

# Now, add this to your main script after you get the 'predictions'
print("\nCalculating Precision and Recall...")
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)

# Average precision and recall
avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
avg_recall = sum(rec for rec in recalls.values()) / len(recalls)

print(f"Average Precision@10: {avg_precision:.4f}")
print(f"Average Recall@10: {avg_recall:.4f}")


Calculating Precision and Recall...
Average Precision@10: 0.0000
Average Recall@10: 0.0000


# Hyperparameter tunning using grid search

In [33]:
!cd

c:\machihe learning\Projects\Recomendation system\notebooks


In [34]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy
from collections import defaultdict
import os

# --- 1. Load and Prepare the Data ---
try:
    # Use a relative path that works on any machine
    interactions_df = pd.read_csv('C:\machihe learning\Projects\Recomendation system\src\interactions.csv')
    interactions_df['rating'] = pd.to_numeric(interactions_df['rating'])
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: The 'interactions.csv' file not found. Make sure it is in the same directory as this script.")

# --- 2. Data Formatting for Surprise Library ---
reader = Reader(rating_scale=(1, 5)) 
data = Dataset.load_from_df(interactions_df[['user_id', 'product_id', 'rating']], reader)

# --- 3. Hyperparameter Tuning ---
# 'cv=5' means we will use 5-fold cross-validation
# 'n_jobs=-1' uses all available CPU cores for faster execution
param_grid = {
    'n_epochs': [20, 30],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.05]
}

print("\nStarting GridSearchCV for SVD model...")
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs.fit(data)

best_params = gs.best_params['rmse']
print(f"Best RMSE score: {gs.best_score['rmse']:.4f}")
print(f"Best hyperparameters: {best_params}")

# --- 4. Final Model Training and Evaluation with Best Parameters ---
# Split the data into a training set and a testing set for evaluation
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Use the best hyperparameters found by GridSearchCV
final_model = SVD(
    n_epochs=best_params['n_epochs'],
    lr_all=best_params['lr_all'],
    reg_all=best_params['reg_all']
)

# Train the final model on the training set
final_model.fit(trainset)

# Make predictions on the testing set
predictions = final_model.test(testset)

# --- 5. Final Evaluation with Precision and Recall ---
def precision_recall_at_k(predictions, k=10, threshold=4):
    """
    Return precision and recall at k metrics for each user.
    """
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = defaultdict(float)
    recalls = defaultdict(float)
    
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        n_relevant = sum(1 for _, true_r in user_ratings if true_r >= threshold)
        n_rec_k = k
        n_rel_and_rec_k = sum(1 for est, true_r in user_ratings[:k] if est >= threshold and true_r >= threshold)

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_relevant if n_relevant != 0 else 0

    return precisions, recalls

print("\nCalculating Final Precision and Recall...")
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)

avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
avg_recall = sum(rec for rec in recalls.values()) / len(recalls)

print(f"Average Precision@10 (Tuned Model): {avg_precision:.4f}")
print(f"Average Recall@10 (Tuned Model): {avg_recall:.4f}")

Data loaded successfully.

Starting GridSearchCV for SVD model...
Best RMSE score: 1.3658
Best hyperparameters: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.05}

Calculating Final Precision and Recall...
Average Precision@10 (Tuned Model): 0.0000
Average Recall@10 (Tuned Model): 0.0000


In [36]:
# --- Correctly instantiate the final model with best params ---
final_model = SVD(
    n_epochs=20,
    lr_all=0.005,
    reg_all=0.05
)

# --- Split, Train, and Predict ---
reader = Reader(rating_scale=(1, 5)) 
data = Dataset.load_from_df(interactions_df[['user_id', 'product_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
final_model.fit(trainset)
predictions = final_model.test(testset)

# --- Final Evaluation with Corrected Precision and Recall Logic ---
def precision_recall_at_k(predictions, k=10, threshold=3):
    """
    Return precision and recall at k metrics for each user.
    Lowering the threshold to 3 to include 'add to cart' events.
    """
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = defaultdict(float)
    recalls = defaultdict(float)
    
    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        
        n_relevant = sum(1 for _, true_r in user_ratings if true_r >= threshold)
        n_rec_k = k
        n_rel_and_rec_k = sum(1 for est, true_r in user_ratings[:k] if est >= threshold and true_r >= threshold)

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_relevant if n_relevant != 0 else 0

    return precisions, recalls

print("\nCalculating Final Precision and Recall...")
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=3)

avg_precision = sum(prec for prec in precisions.values()) / len(precisions)
avg_recall = sum(rec for rec in recalls.values()) / len(recalls)

print(f"Average Precision@10 (Tuned Model): {avg_precision:.4f}")
print(f"Average Recall@10 (Tuned Model): {avg_recall:.4f}")


Calculating Final Precision and Recall...
Average Precision@10 (Tuned Model): 0.0008
Average Recall@10 (Tuned Model): 0.0010


In [38]:
# --- Save the trained model to a file ---
# This is a critical production-ready step
from surprise import dump
model_path = os.path.join('data', 'processed', 'final_model.pkl')
dump.dump(model_path, algo=final_model)
print(f"Final model saved to: {model_path}")

Final model saved to: data\processed\final_model.pkl


In [39]:
# --- Create a Cold-Start Fallback (Popularity-Based Recommender) ---
# Group by product and count interactions to find the most popular products
popular_products = interactions_df.groupby('product_id')['rating'].count().sort_values(ascending=False).head(10).index.tolist()
popular_products_info = products_df[products_df['product_id'].isin(popular_products)]['product_name'].tolist()


In [43]:
def get_recommendations(user_id, model, trainset, all_products, top_n=10):
    # Check if the user is in our training data
    if trainset.knows_user(user_id):
        print("User found in training data. Using collaborative filtering.")
        
        # Get products the user has NOT rated by checking all products
        # This is a more robust way to handle unseen items
        unseen_products = []
        for product_id in all_products:
            if not trainset.knows_item(product_id) or not trainset.knows_user(user_id) or not trainset.knows_item(trainset.to_inner_iid(product_id)) or (user_id, product_id) not in trainset._inner_id_to_raw_id_item[trainset.to_inner_iid(product_id)]:
                unseen_products.append(product_id)

        # Get predictions for unseen items
        predictions = [model.predict(user_id, product_id) for product_id in unseen_products]
        predictions.sort(key=lambda x: x.est, reverse=True)
        top_recommendations = [pred.iid for pred in predictions[:top_n]]

    else:
        # Fallback to popularity-based for new users
        print("New user. Using popularity-based recommendations.")
        # Assuming you've already calculated 'popular_products' from the full dataset
        # popular_products = interactions_df.groupby('product_id')['rating'].count().sort_values(ascending=False).head(10).index.tolist()
        top_recommendations = popular_products

    return top_recommendations

In [44]:
# --- Test the recommendation function ---
all_products_ids = products_df['product_id'].unique()

In [45]:
# 1. Get recommendations for a known user (e.g., user 1)
known_user_id = interactions_df['user_id'].iloc[0]
recs_known_user = get_recommendations(known_user_id, final_model, all_products_ids)
print(f"\nRecommendations for known user {known_user_id}: {recs_known_user}")

TypeError: get_recommendations() missing 1 required positional argument: 'all_products'